In [36]:
import pandas as pd
from datetime import timedelta
import glob
import os
import matplotlib.pyplot as plt
import numpy as np

In [37]:
df = pd.read_csv(r"C:\Users\Administrator\Desktop\raphi_other\repositories\ts\data/raw/blood_glucose/train.csv")
#ts.plot()




C:\Users\Administrator\AppData\Local\Temp\ipykernel_15108\2568112746.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Administrator\Desktop\raphi_other\repositories\ts\data/raw/blood_glucose/train.csv")


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: float64(433), object(75)
memory usage: 686.1+ MB


In [40]:
# Function to downcast float64 to float32
def downcast_float64_to_float32_and_objects_to_categories(df):
    # Downcast float64 columns to float32
    float64_cols = df.select_dtypes(include=['float64']).columns
    df[float64_cols] = df[float64_cols].astype(np.float32)
    
    # Convert object columns to category
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].astype('category')
    
    
    return df


df = downcast_float64_to_float32_and_objects_to_categories(df)

In [135]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
import joblib 
from tensorflow.keras.callbacks import EarlyStopping
# Directory to save the trained autoencoders and RandomForest model
AUTOENCODER_SAVE_DIR = r"../encoders"
MODEL_SAVE_DIR = "../models"
os.makedirs(AUTOENCODER_SAVE_DIR, exist_ok=True)
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)


# Before the autoencoding loop
columns_activities = ['activity-5:55', 'activity-5:50', 'activity-5:45', 'activity-5:40',
   'activity-5:35', 'activity-5:30', 'activity-5:25', 'activity-5:20',
   'activity-5:15', 'activity-5:10', 'activity-5:05', 'activity-5:00',
   'activity-4:55', 'activity-4:50', 'activity-4:45', 'activity-4:40',
   'activity-4:35', 'activity-4:30', 'activity-4:25', 'activity-4:20',
   'activity-4:15', 'activity-4:10', 'activity-4:05', 'activity-4:00',
   'activity-3:55', 'activity-3:50', 'activity-3:45', 'activity-3:40',
   'activity-3:35', 'activity-3:30', 'activity-3:25', 'activity-3:20',
   'activity-3:15', 'activity-3:10', 'activity-3:05', 'activity-3:00',
   'activity-2:55', 'activity-2:50', 'activity-2:45', 'activity-2:40',
   'activity-2:35', 'activity-2:30', 'activity-2:25', 'activity-2:20',
   'activity-2:15', 'activity-2:10', 'activity-2:05', 'activity-2:00',
   'activity-1:55', 'activity-1:50', 'activity-1:45', 'activity-1:40',
   'activity-1:35', 'activity-1:30', 'activity-1:25', 'activity-1:20',
   'activity-1:15', 'activity-1:10', 'activity-1:05', 'activity-1:00',
   'activity-0:55', 'activity-0:50', 'activity-0:45', 'activity-0:40',
   'activity-0:35', 'activity-0:30', 'activity-0:25', 'activity-0:20',
   'activity-0:15', 'activity-0:10', 'activity-0:05', 'activity-0:00']

columns_to_encode = ['p_num', 'time']

In [137]:
df.head()


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [194]:
def autoencode_columns_999(df, column_indices, encoding_dim=5, epochs=3, batch_size=32, autoencoder_id=0, train=True):
    print(f"Autoencoding columns from indices {column_indices[0]} to {column_indices[-1]} (feature set {autoencoder_id+1})")

    # Ensure that the number of columns (range) is exactly 216
    assert (column_indices[-1] - column_indices[0] + 1) == 216, \
        f"Expected 216 features (from column_indices {column_indices[0]} to {column_indices[-1]}), but got {(column_indices[-1] - column_indices[0] + 1)} columns. Please check your column_indices."

    # Extract the subset of columns based on indices
    data_subset = df.iloc[:, column_indices].values
    print(f"Original data shape for feature set {autoencoder_id+1}: {data_subset.shape}")
    
    # Impute missing values with the mean of the column
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data_subset)

    # Standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_imputed)
    
    if train:
        # Autoencoder architecture
        input_dim = data_subset.shape[1]  # Dynamically setting input dimensions (should be 216)
        print(f"Autoencoder input dimension: {input_dim}")

        input_layer = Input(shape=(input_dim,))
        encoded = Dense(encoding_dim, activation='relu', 
                        activity_regularizer=regularizers.l1(1e-5))(input_layer)
        decoded = Dense(input_dim, activation='sigmoid')(encoded)

        # Define the autoencoder model
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer='adam', loss='mse')

        # Train the autoencoder
        print(f"Training autoencoder {autoencoder_id}...")
        autoencoder.fit(data_scaled, data_scaled,
                        epochs=epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        verbose=1,
                    #    validation_data=(X_val, X_val),
                        callbacks=[EarlyStopping(monitor='train_loss', mode='min', patience=5)])

        # Save the trained autoencoder, scaler, and imputer
        autoencoder.save(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{autoencoder_id}.keras"))
        joblib.dump(scaler, os.path.join(AUTOENCODER_SAVE_DIR, f"scaler_{autoencoder_id}.pkl"))
        joblib.dump(imputer, os.path.join(AUTOENCODER_SAVE_DIR, f"imputer_{autoencoder_id}.pkl"))
        print(f"Autoencoder {autoencoder_id}, scaler, and imputer saved.")

        # Create encoder model to extract the compressed representation
        encoder = Model(inputs=input_layer, outputs=encoded)
    else:
        # Load the trained autoencoder components for test data
        print(f"Loading autoencoder {autoencoder_id} for test data transformation...")
        autoencoder = load_model(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{autoencoder_id}.keras"))
        scaler = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"scaler_{autoencoder_id}.pkl"))
        imputer = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"imputer_{autoencoder_id}.pkl"))
        
        # Impute and scale test data
        data_imputed = imputer.transform(data_subset)
        data_scaled = scaler.transform(data_imputed)

        # Use only the encoder part of the autoencoder
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=1).output)

    # Generate reduced data
    reduced_data = encoder.predict(data_scaled)
    print(f"Reduced data shape for feature set {autoencoder_id}: {reduced_data.shape}")

    # Create a DataFrame for the reduced features
    reduced_df = pd.DataFrame(reduced_data, 
                              columns=[f"autoencoded_{autoencoder_id}_{i}" for i in range(encoding_dim)])
    
    return reduced_df


In [196]:
def autoencode_columns(df, column_indices, encoding_dim=5, epochs=3, batch_size=32, autoencoder_id=0, train=True):
    print(f"Autoencoding columns")

 
    #print(f"Autoencoding columns {column_indices.start}-{column_indices.stop-1} (feature set {autoencoder_id+1})")

    # Extract the subset of columns
    data_subset = df.iloc[:, column_indices].values
    #print(f"Original data shape for feature set {autoencoder_id+1}: {data_subset.shape}")
    
    # Impute missing values with the mean of the column
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data_subset)

    # Standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_imputed)
    
    if train:
        print("SIZE SIZE")
        print(data_subset.shape[1])
        # Autoencoder architecture
        input_dim = data_subset.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(encoding_dim, activation='relu', 
                        activity_regularizer=regularizers.l1(1e-5))(input_layer)
        decoded = Dense(input_dim, activation='sigmoid')(encoded)

        # Define the autoencoder model
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer='adam', loss='mse')

        # Train the autoencoder
        print(f"Training autoencoder {autoencoder_id}...")
        autoencoder.fit(data_scaled, data_scaled,
                        epochs=epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        verbose=1,
                        #validation_data=(X_val, X_val),
                        callbacks=[EarlyStopping(monitor='train_loss', mode='min', patience=5)])


        # Save the trained autoencoder, scaler, and imputer
        autoencoder.save(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{autoencoder_id}.keras"))
        joblib.dump(scaler, os.path.join(AUTOENCODER_SAVE_DIR, f"scaler_{autoencoder_id}.pkl"))
        joblib.dump(imputer, os.path.join(AUTOENCODER_SAVE_DIR, f"imputer_{autoencoder_id}.pkl"))
        print(f"Autoencoder {autoencoder_id}, scaler, and imputer saved.")

        # Create encoder model to extract the compressed representation
        encoder = Model(inputs=input_layer, outputs=encoded)
    else:
        # Load the trained autoencoder components for test data
        print(f"Loading autoencoder {autoencoder_id} for test data transformation...")
        autoencoder = load_model(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{autoencoder_id}.keras"))
        scaler = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"scaler_{autoencoder_id}.pkl"))
        imputer = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"imputer_{autoencoder_id}.pkl"))
        
        # Use only the encoder part of the autoencoder
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=1).output)

    # Generate reduced data
    reduced_data = encoder.predict(data_scaled)
    print(f"Reduced data shape for feature set {autoencoder_id}: {reduced_data.shape}")

    # Create a DataFrame for the reduced features
    reduced_df = pd.DataFrame(reduced_data, 
                              columns=[f"autoencoded_{autoencoder_id}_{i}" for i in range(encoding_dim)])
    
    return reduced_df


In [198]:
import category_encoders as ce

def encode_baseN(df, cols, base=3):
    print("base N encoding")
    # Instantiate the encoder
    encoder = ce.BaseNEncoder(cols=cols, return_df=True, base=base)
    # Fit and transform the data
    df_encoded = encoder.fit_transform(df[cols])
    return df_encoded, encoder


def train_pipeline(df, target_col='bg+1:00'):
    print("Starting training pipeline...")
    # Drop 'id' and target columns
    X = df.drop(columns=['id', target_col])  # Features
    y = df[target_col]  # Target

    print("Splitting data into train/test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=True)
    print(f"Train set size: {X_train.shape}, Test set size: {X_test.shape}")


    #### changing to whole dataset!
    X_train = X
    y_train = y
    

    # Check how many columns you have for autoencoding
    total_columns = X_train.shape[1]
    print(f"Total columns to autoencode: {total_columns}")
    
    column_start = 0  # Adjust this if needed based on your column structure
    column_batch_size = 72
    encoding_dim = 5  # Dimensionality reduction target

    import joblib

    # Base-N encode 'p_num' and 'time' columns
    columns_to_encode = ['p_num', 'time']
    X_train_encoded_baseN, baseN_encoder = encode_baseN(X_train, columns_to_encode)
    X_test_encoded_baseN = baseN_encoder.transform(X_test[columns_to_encode])
    
    # Convert to DataFrame
    X_train_encoded_full = pd.DataFrame(X_train_encoded_baseN)
    X_test_encoded_full = pd.DataFrame(X_test_encoded_baseN)
    
    # Save the BaseNEncoder
    encoder_save_path = r'../encoders/baseN_encoder.pkl'
    joblib.dump(baseN_encoder, encoder_save_path)
    print(f"BaseNEncoder saved to {encoder_save_path}")

    X_train_encoded_full = pd.DataFrame(X_train_encoded_baseN)
    X_test_encoded_full = pd.DataFrame(X_test_encoded_baseN)

    print("X_train_encoded_full after adding encoded p_num and time: ")
    print(X_train_encoded_full.shape)
    #print(X_train_encoded_full.head)


    ### ----------------- ADDINFG THE FIRST ACTIVITY ENCODER BASENEncoder .-------------------


    # Base-N encode the activity columns first
    X_train_encoded_activities, baseN_encoder_activities = encode_baseN(X_train, columns_activities)
    X_test_encoded_activities = baseN_encoder_activities.transform(X_test[columns_activities])
    
    # Save the BaseNEncoder for activity columns
    encoder_activities_save_path = r'../encoders/baseN_encoder_activities.pkl'
    joblib.dump(baseN_encoder_activities, encoder_activities_save_path)
    print(f"BaseNEncoder for activities saved to {encoder_activities_save_path}")

    # Drop 'p_num' and 'time' columns (assuming they should not be in the encoded activity data)
    X_train_encoded_activities = pd.DataFrame(X_train_encoded_activities).drop(columns=columns_to_encode, errors='ignore')
    X_test_encoded_activities = pd.DataFrame(X_test_encoded_activities).drop(columns=columns_to_encode, errors='ignore')

    print("base n encoded stuff")
    print(X_train_encoded_activities.shape)

    ### ----------------- ADDINFG THE FIRST ACTIVITY ENCODER BASENEncoder AND AUTOENCODER.-------------------
    
    column_indices_activities = list(range(0, 216)) #list(range(X_train_encoded_activities.shape[1]))  # Autoencode all activity columns
    print("indixes")
    print(column_indices_activities)
    
    # Autoencode the Base-N encoded activity columns
    X_train_autoencoded_activities = autoencode_columns_999(X_train_encoded_activities, column_indices_activities, encoding_dim=encoding_dim, autoencoder_id=999)
    X_test_autoencoded_activities = autoencode_columns_999(X_test_encoded_activities, column_indices_activities, encoding_dim=encoding_dim, autoencoder_id=999, train=False)

    print("autoencoded activities shape")
    print(X_train_autoencoded_activities.shape)
    
    #--------------------------------------------------------------------#--------------------------------------------------------------------
    # Concatenate the autoencoded columns back into the full dataset
    X_train_encoded_full = pd.concat([X_train_encoded_full.reset_index(drop=True), 
                                  pd.DataFrame(X_train_autoencoded_activities).reset_index(drop=True)], axis=1)
    X_test_encoded_full = pd.concat([X_test_encoded_full.reset_index(drop=True), 
                                 pd.DataFrame(X_test_autoencoded_activities).reset_index(drop=True)], axis=1)

    X_train = X_train.drop(columns=columns_activities + columns_to_encode)
    X_test = X_test.drop(columns=columns_activities + columns_to_encode)

    print("shape full before auto-encoding:")
    print(X_train_encoded_full.shape)

    #---------------------------------------------------------------------------------------------------------------------
    print("shape x_train before auto-encoding:")
    print(X_train.shape)
    #print(X_train.head)

    
    # Loop through the columns in batches and autoencode
    for i in range(6):
        
        column_end = column_start + column_batch_size
        print("column end")
        print(column_end)
        
        if column_end > total_columns:
            print(f"Not enough columns to process for batch {i}, stopping at {column_start} to {total_columns}")
            break  # Stop if there aren't enough columns

        column_indices = range(column_start, column_end)
        print(f"Processing feature set {i} (columns {column_start}-{column_end+1} + 1)")

        # Autoencode the training data
        X_train_encoded = autoencode_columns(X_train, column_indices, encoding_dim=encoding_dim, autoencoder_id=i)

        # Use the same autoencoder for test data
        X_test_encoded = autoencode_columns(X_test, column_indices, encoding_dim=encoding_dim, autoencoder_id=i, train=False)

        # Replace the original columns with the encoded columns
        X_train_encoded_full = pd.concat([X_train_encoded_full, X_train_encoded], axis=1)
        X_test_encoded_full = pd.concat([X_test_encoded_full, X_test_encoded], axis=1)

        column_start = column_end  # Move to the next batch of columns
        print("x-train-full after ALL ENCODINGS:")
        print(X_train_encoded_full.shape)


    print("All features autoencoded. Training Random Forest Regressor...")
    #Append the Base-N encoded columns back here to X_train and X_test

    from xgboost import XGBRegressor

    # xgb = XGBClassifier(
#     n_estimators=600,
#     learning_rate=0.2669112505018992,
#     max_depth=5,
#     random_state=42,
#     reg_lambda=1.2259716591605452,
#     subsample=0.704976942819638,
#     colsample_bytree=0.9,
#     min_child_weight=4,
#     alpha= 0.14170716330946964,    # Added L1 regularization
#     eval_metric='mlogloss',  # Consider custom loss for ordinal
#     objective='multi:softmax',  # Using softmax but can tweak for ordinal
#     num_class=3  # Assuming 3 ordinal classes
# )
    xgb_params = {
        'n_estimators': 400,       # Number of boosting rounds
        'max_depth': 5,           # Maximum depth of the trees
        'learning_rate': 0.2669112505018992,      # Step size shrinkage
        'subsample': 0.704976942819638,          # Subsampling of rows
        'colsample_bytree': 0.9,   # Subsampling of columns
        'min_child_weight': 4,     # Minimum sum of instance weight
        'gamma': 0,                # Minimum loss reduction required to make a split
        'alpha': 7.455101924325866,              # L1 regularization term (equivalent to lambda_l1)
        'lambda': 0.007915220427757011,               # L2 regularization term (equivalent to lambda_l2)
        'random_state': 42         # Ensures reproducibility
    }
    
    # Train XGBoost Regressor with parameters
    xgb_regressor = XGBRegressor(**xgb_params)
    # Train Random Forest
    
    print("in forrest shape x_train: ")
    print(X_train_encoded_full.shape)
    print(y_train.shape)
    
    rf_regressor.fit(X_train_encoded_full, y_train)
    print("Random Forest training complete.")
    
    # Save the trained Random Forest model
    joblib.dump(rf_regressor, os.path.join(MODEL_SAVE_DIR, "random_forest_model_new.pkl"))
    print("Random Forest model saved.")
    
    # Predict and evaluate on test set
    #print("Making predictions on the test set...")
    #y_pred = rf_regressor.predict(X_test_encoded_full)
    #rmse = sqrt(mean_squared_error(y_test, y_pred))
    #print(f"RMSE: {rmse}")

# Run the training pipeline
# Assuming df is your training dataset

train_pipeline(df)

Starting training pipeline...
Splitting data into train/test sets...
Train set size: (141619, 506), Test set size: (35405, 506)
Total columns to autoencode: 506
base N encoding
BaseNEncoder saved to ../encoders/baseN_encoder.pkl
X_train_encoded_full after adding encoded p_num and time: 
(177024, 9)
base N encoding
BaseNEncoder for activities saved to ../encoders/baseN_encoder_activities.pkl
base n encoded stuff
(177024, 216)
indixes
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `train_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.9710
Epoch 3/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.9633
Autoencoder 999, scaler, and imputer saved.
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 5s 813us/step
Reduced data shape for feature set 999: (177024, 5)
Autoencoding columns from indices 0 to 215 (feature set 1000)
Original data shape for feature set 1000: (35405, 216)
Loading autoencoder 999 for test data transformation...
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 1s 836us/step
Reduced data shape for feature set 999: (35405, 5)
autoencoded activities shape
(177024, 5)
shape full before auto-encoding:
(177024, 14)
shape x_train before auto-encoding:
(177024, 432)
column end
72
Processing feature set 0 (columns 0-73 + 1)
Autoencoding columns
SIZE SIZE
72
Training autoencoder 0...
Epoch 1/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.7264
Epoch 2/3
 164/5532 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.6075  

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `train_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 989us/step - loss: 0.5981
Epoch 3/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 997us/step - loss: 0.5901 
Autoencoder 0, scaler, and imputer saved.
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 5s 825us/step
Reduced data shape for feature set 0: (177024, 5)
Autoencoding columns
Loading autoencoder 0 for test data transformation...
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Reduced data shape for feature set 0: (35405, 5)
x-train-full after ALL ENCODINGS:
(177024, 19)
column end
144
Processing feature set 1 (columns 72-145 + 1)
Autoencoding columns
SIZE SIZE
72
Training autoencoder 1...
Epoch 1/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 1.0090
Epoch 2/3
 130/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.9678

...

5532/5532 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.6699
Epoch 3/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.6575
Autoencoder 3, scaler, and imputer saved.
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 4s 801us/step
Reduced data shape for feature set 3: (177024, 5)
Autoencoding columns
Loading autoencoder 3 for test data transformation...
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 1s 890us/step
Reduced data shape for feature set 3: (35405, 5)
x-train-full after ALL ENCODINGS:
(177024, 34)
column end
360
Processing feature set 4 (columns 288-361 + 1)
Autoencoding columns
SIZE SIZE
72
Training autoencoder 4...
Epoch 1/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 0.9308
Epoch 2/3
 136/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.8579

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `train_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.8313
Epoch 3/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.8234
Autoencoder 4, scaler, and imputer saved.
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 998us/step
Reduced data shape for feature set 4: (177024, 5)
Autoencoding columns
Loading autoencoder 4 for test data transformation...
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 1s 834us/step
Reduced data shape for feature set 4: (35405, 5)
x-train-full after ALL ENCODINGS:
(177024, 39)
column end
432
Processing feature set 5 (columns 360-433 + 1)
Autoencoding columns
SIZE SIZE
72
Training autoencoder 5...
Epoch 1/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.8040
Epoch 2/3
 136/5532 ━━━━━━━━━━━━━━━━━━━━ 4s 926us/step - loss: 0.6923

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `train_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.6860
Epoch 3/3
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.6749
Autoencoder 5, scaler, and imputer saved.
5532/5532 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
Reduced data shape for feature set 5: (177024, 5)
Autoencoding columns
Loading autoencoder 5 for test data transformation...
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Reduced data shape for feature set 5: (35405, 5)
x-train-full after ALL ENCODINGS:
(177024, 44)
All features autoencoded. Training Random Forest Regressor...
in forrest shape x_train: 
(177024, 44)
(177024,)
Random Forest training complete.
Random Forest model saved.


## Prediction

In [184]:
def load_encoder(model_path):
    """
    Load the autoencoder and return the encoder part of the model.

    Args:
    - model_path: Path to the saved autoencoder model (.keras or .h5 file).

    Returns:
    - encoder: The encoder model that outputs the reduced dimensionality.
    """
    # Load the full autoencoder
    autoencoder = load_model(model_path)
    
    # Print the summary of the full autoencoder model
    print("Autoencoder Model Summary:")
    autoencoder.summary()

    # Extract the encoder part (from input layer to the bottleneck layer)
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[1].output)

    # Print the summary of the encoder
    print("\nEncoder Model Summary:")
    encoder.summary()

    return encoder

In [186]:
import os
from keras.models import load_model

def check_autoencoder_input_shape(model_path):
    """
    Load the autoencoder model and print its input shape.

    Args:
    - model_path: Path to the saved autoencoder model (.keras or .h5 file).
    """
    # Load the autoencoder model
    autoencoder = load_model(model_path)

    # Print the model summary to inspect the architecture and input shape
    print("Autoencoder Model Summary:")
    autoencoder.summary()

    # Check the input layer's configuration
    input_layer = autoencoder.layers[0]  # Input layer is usually the first layer
    #input_shape = input_layer.input_shape

    print(f"\nExpected input shape: {input_layer}")
    return input_layer

# Example usage:
model_path = os.path.join(r'../encoders', 'autoencoder_999.keras')  # Adjust this path to your model
input_shape = check_autoencoder_input_shape(model_path)
#input_shape

Autoencoder Model Summary:


Model: "functional_140"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)          │ (None, 216)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 5)                   │           1,085 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 216)                 │           1,296 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,145 (27.91 KB)

 Trainable params: 2,381 (9.30 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,764 (18.61 KB)


Expected input shape: <InputLayer name=input_layer_30, built=True>


In [188]:
#print("Loading BaseNEncoder for activity columns...")
#    baseN_encoder_activities = joblib.load(r'../encoders/baseN_encoder_activities.pkl')


In [200]:
test_df = pd.read_csv(r"C:\Users\Administrator\Desktop\raphi_other\repositories\ts\data/raw/blood_glucose/test.csv")

In [202]:
# Load the test dataset (assuming test_df is the loaded DataFrame for test.csv)
test_df = pd.read_csv(r'../data/raw/blood_glucose/test.csv')

# Run the prediction pipeline
predictions = prediction_pipeline(test_df, MODEL_SAVE_DIR, columns_to_encode, columns_activities)

# Prepare the submission file
print("Preparing submission file...")
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'Id' is a column in test.csv
    'bg+1:00': predictions
})


# Save the submission to CSV
submission_df.to_csv(r'../results/sub3.csv', index=False)

Starting prediction pipeline...
Loading BaseNEncoder for 'p_num' and 'time'...
BaseN encoding 'p_num' and 'time'...
(3644, 9)
Loading BaseNEncoder for activity columns...
BaseN encoding activity columns...
(3644, 216)
After creating dataframe for the encoded activities WITHOUT AE
(3644, 216)
Loading autoencoder for activity columns...
Autoencoder Model Summary:


Model: "functional_168"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_37 (InputLayer)          │ (None, 216)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 5)                   │           1,085 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 216)                 │           1,296 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,145 (27.91 KB)

 Trainable params: 2,381 (9.30 KB)

...

 Trainable params: 365 (1.43 KB)

 Non-trainable params: 0 (0.00 B)

Imputing missing values in test data for feature set 5...
Standardizing test data for feature set 5...
  1/114 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step

C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Reduced test data shape for feature set 5: (3644, 5)
Loading Random Forest model...
Making predictions on the test set...
Predictions complete.
Preparing submission file...


## Helpers

In [102]:
def prediction_pipeline(test_df, model_path, columns_to_encode, columns_activities, num_features=6, encoding_dim=5):
    """
    Prediction pipeline for processing test data and predicting results using a trained Random Forest model.
    Args:
    - test_df: The test DataFrame that needs to be processed.
    - model_path: Path to the directory where the Random Forest model is saved.
    - columns_to_encode: The list of first two columns to be Base-N encoded.
    - columns_activities: The list of activity columns to be Base-N encoded and then autoencoded.
    - num_features: The number of feature sets to be autoencoded.
    - encoding_dim: The dimensionality of the reduced features from the autoencoders.

    Returns:
    - predictions: Predictions made on the transformed test data using the loaded Random Forest model.
    """
    #test_df.columns[75:147]
    test_df_final = pd.DataFrame()
    print("Starting prediction pipeline...")

    ### Step 1: Base-N encode the first two columns ('p_num' and 'time')
    print("Loading BaseNEncoder for 'p_num' and 'time'...")
    baseN_encoder = joblib.load(r'../encoders/baseN_encoder.pkl')
    
    print("BaseN encoding 'p_num' and 'time'...")
    X_test_encoded_baseN = baseN_encoder.transform(test_df[columns_to_encode])
    print(X_test_encoded_baseN.shape)
    
    # Convert to DataFrame and concatenate with test_df
    X_test_encoded_baseN_df = pd.DataFrame(X_test_encoded_baseN)


    ##################################### ADDING HERE THe first to final
    test_df_final = pd.concat([test_df_final, X_test_encoded_baseN_df], axis=1)
    
    ### Step 2: Base-N encode the activity columns
    print("Loading BaseNEncoder for activity columns...")
    baseN_encoder_activities = joblib.load(r'../encoders/baseN_encoder_activities.pkl')
    
    print("BaseN encoding activity columns...")
    X_test_encoded_activities = baseN_encoder_activities.transform(test_df[columns_activities])
    print(X_test_encoded_activities.shape)
    
    # Convert to DataFrame
    X_test_encoded_activities_df = pd.DataFrame(X_test_encoded_activities)

    print("After creating dataframe for the encoded activities WITHOUT AE")
    print(X_test_encoded_activities_df.shape)
    
    # Step 3: Autoencode the Base-N encoded activity columns
    print("Loading autoencoder for activity columns...")

    #autoencoder_activities = load_model(os.path.join(AUTOENCODER_SAVE_DIR, "autoencoder_999.keras"))
    # Load the encoder
    autoencoder_activities = load_encoder(os.path.join(AUTOENCODER_SAVE_DIR, "autoencoder_999.keras"))


    print("REDUCED activities AE the Base-N encoded activity columns...")
    reduced_activities = autoencoder_activities.predict(X_test_encoded_activities_df)
    print(reduced_activities)

    print("testo")
    reduced_activities_df = pd.DataFrame(reduced_activities, 
                                         columns=[f"autoencoded_999_{i}" for i in range(5)])
    print("testo2")
    print("REDUCED reduced_activities_df")
    print(reduced_activities_df.shape)
    
    # Concatenate autoencoded activity columns with the test DataFrame
    test_df_final = pd.concat([test_df_final, reduced_activities_df], axis=1)

    ### Step 4: Iterate over 6 feature sets (each 72 columns long) and apply autoencoders
    column_start = 3  # Starting position of the first feature set (after the first 3 columns)
    
    selected_columns = df.columns[column_start:column_start+72]
    print("FEATURES")
    print(num_features)
    
    # --------------------------------------------------------------------------------------------------------------------------------------------

    for i in range(num_features):
        
        column_end = column_start + 72  # Each feature set is 72 columns long
        column_indices = range(column_start, column_end)
        print(column_indices)
        print(test_df.columns[column_indices].tolist())


        # Print the selected columns
        #for idx, col in enumerate(selected_columns, start=column_start):
         #   print(f"Index {idx}: {col}")

        print(f"Loading autoencoder, scaler, and imputer for feature set {i} (columns {column_start}-{column_end-1})")

        # Load the autoencoder, scaler, and imputer for this feature set
        #autoencoder = load_model(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{i}.keras"))
        autoencoder = load_encoder(os.path.join(AUTOENCODER_SAVE_DIR, f"autoencoder_{i}.keras"))
        scaler = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"scaler_{i}.pkl"))
        imputer = joblib.load(os.path.join(AUTOENCODER_SAVE_DIR, f"imputer_{i}.pkl"))

        # Impute missing values in the test data
        print(f"Imputing missing values in test data for feature set {i}...")
        data_imputed = imputer.transform(test_df.iloc[:, column_indices])

        # Standardize the test data
        print(f"Standardizing test data for feature set {i}...")
        data_scaled = scaler.transform(data_imputed)

        # Autoencode the test data
        reduced_data = autoencoder.predict(data_scaled)
        print(f"Reduced test data shape for feature set {i}: {reduced_data.shape}")

        # Create a DataFrame for the reduced features
        reduced_df = pd.DataFrame(reduced_data, 
                                  columns=[f"autoencoded_{i}_{j}" for j in range(encoding_dim)])

        # Replace original columns with autoencoded columns
        #test_df = pd.concat([test_df.drop(columns=test_df.columns[column_indices]), reduced_df], axis=1)
        test_df_final = pd.concat([test_df_final, reduced_df], axis=1)

        #column_start = column_end  # Move to the next set of columns

    ### Step 5: Load the trained RandomForest model and make predictions
    print("Loading Random Forest model...")
    rf_regressor = joblib.load(os.path.join(model_path, "random_forest_model.pkl"))

    print("Making predictions on the test set...")
    predictions = rf_regressor.predict(test_df_final)
    print("Predictions complete.")

    return predictions
